In [1]:
import json
from landsatxplore.api import API
import getpass
import pandas as pd
import geopandas as gpd

In [2]:
username = input('Enter your username')
password = getpass.getpass('Enter your password')

Enter your username sgoliber
Enter your password ········


In [3]:
bbox = gpd.read_file('bbox/test_bbox.shp')
bounds = bbox.bounds.iloc[0].to_list()

In [13]:
# Initialize a new API instance
api = API(username, password)

# Perform a request
response = api.request(endpoint="dataset-catalogs")
print(response)

# Search for Landsat TM scenes
scenes = api.search(
    dataset='landsat_ot_c2_l2',
    bbox = bounds,
    start_date='2022-10-01',
    end_date='2022-12-31',
    max_cloud_cover=5
)

# log out
api.logout()

{'EE': 'EarthExplorer', 'GV': 'GloVis', 'HDDS': 'HDDS Explorer'}


In [14]:
# Create a DataFrame from the scenes
df_scenes = pd.DataFrame(scenes)
df_scenes = df_scenes[['display_id','wrs_path', 'wrs_row','satellite','cloud_cover','acquisition_date']]
df_scenes.sort_values('acquisition_date', ascending=False, inplace=True)
df_scenes

,display_id,wrs_path,wrs_row,satellite,cloud_cover,acquisition_date
0,LC09_L2SP_009010_20221005_20230326_02_T1,9,10,9,4,2022-10-05
1,LC08_L2SP_010011_20221004_20221012_02_T1,10,11,8,3,2022-10-04
2,LC09_L2SP_011010_20221003_20230327_02_T1,11,10,9,1,2022-10-03


In [9]:
from landsatxplore.earthexplorer import EarthExplorer
import os

# Initialize the API
ee = EarthExplorer(username, password)

# Select the first scene
ID = 'LC08_L2SP_010010_20221004_20221012_02_T1'

# Download the scene 
try: 
    ee.download(ID, output_dir='./data')
    print('{} succesful'.format(ID))
    
# Additional error handling
except:
    if os.path.isfile('./data/{}.tar'.format(ID)):
        print('{} error but file exists'.format(ID))
    else:
        print('{} error'.format(ID))

ee.logout()

Download failed with dataset id 1 of 3. Re-trying with the next one.


100%|████████████████████████████████████████| 776M/776M [02:39<00:00, 5.09MB/s]


None of the archived ids succeeded! Update necessary!
LC08_L2SP_010010_20221004_20221012_02_T1 error but file exists


In [10]:
import tarfile

# Extract files from tar archive
tar = tarfile.open('./data/{}.tar'.format(ID))
tar.extractall('./data/{}'.format(ID))
tar.close()